In [1]:
import matplotlib
matplotlib.use("TkAgg")
import tqdm
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time
import umap
import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_png(png_files):
    print(f'=== load {png_files} ===')
    files = glob.glob(png_files)
    file_sample = Image.open(files[0]).convert('L')
    png_array = np.empty((len(files), file_sample.width, file_sample.height))
    count = 0
    for file in tqdm.tqdm(files):
        png_array[count] = np.array(Image.open(file).convert('L')).astype('float32') / 255
        count += 1
    return png_array

In [3]:
parm_mp = load_png('data/2021_02_21 US 4クラス/MP掌側/*.png')
dorsal_mp = load_png('data/2021_02_21 US 4クラス/MP背側/*.png')
dorsal_wrist = load_png('data/2021_02_21 US 4クラス/手関節背側/*.png')

  2%|▏         | 7/300 [00:00<00:04, 61.14it/s]

=== load data/2021_02_21 US 4クラス/MP掌側/*.png ===


  3%|▎         | 9/300 [00:00<00:03, 85.28it/s]

=== load data/2021_02_21 US 4クラス/MP背側/*.png ===


  2%|▏         | 7/300 [00:00<00:04, 68.90it/s]

=== load data/2021_02_21 US 4クラス/手関節背側/*.png ===


100%|██████████| 300/300 [00:04<00:00, 67.62it/s]


In [4]:
def show_sample(images):
    num = len(images)
    fig = plt.figure(figsize=(num * 4, 4))
    count = 1
    for title, image in images.items():
        axis = fig.add_subplot(1, num, count)
        plt.title(title)
        axis.imshow(image, cmap='gray', vmin=0, vmax=1)
        count += 1
    plt.show(block=False)

In [5]:
show_sample({
    'parm MP': parm_mp[0],
    'dorsal MP': dorsal_mp[0],
    'dorsal wrist': dorsal_wrist[0]
})

In [6]:
umap_data = np.vstack((
    np.reshape(parm_mp, (parm_mp.shape[0], -1)),
    np.reshape(dorsal_mp, (dorsal_mp.shape[0], -1)),
    np.reshape(dorsal_wrist, (dorsal_wrist.shape[0], -1))
))
umap_label = np.concatenate((
    np.full(parm_mp.shape[0], 1),
    np.full(dorsal_mp.shape[0], 2),
    np.full(dorsal_wrist.shape[0], 3)
))

In [7]:
t1 = time.time()
result3d = umap.UMAP(n_neighbors=5, n_components=3).fit(umap_data)
t2 = time.time()
print(f"経過時間:{t2-t1}")

経過時間:169.81771206855774


In [8]:
x = result3d.embedding_[:, 0]
y = result3d.embedding_[:, 1]
z = result3d.embedding_[:, 2]

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(x, y, z, c=umap_label, s=80, marker=".", cmap='rainbow')
plt.colorbar(sc)
plt.show(block=False)